In [15]:
import numpy as np
from datetime import datetime, timedelta
import re
from dataclasses import dataclass
from pprint import pprint, pformat

In [4]:
@dataclass
class Date:
    month: int
    year: int
    day: int | None = None

    def __post_init__(self):
        if isinstance(self.month, str):
            self.month=int(self.month)
        if isinstance(self.year, str):
            self.year=int(self.year)
        if isinstance(self.day, str):
            self.day=int(self.day)

    def inc_month(self):
        if self.month == 12:
            month = 1
            year = self.year + 1
        else:
            month = self.month + 1
            year = self.year
        return Date(year=year,month=month)

    def dec_month(self):
        if self.month == 1:
            month = 12
            year = self.year - 1
        else:
            month = self.month - 1
            year = self.year
        return Date(year=year,month=month)

    def __hash__(self):
        return hash((self.year, self.month, self.day))

    def to_year_month_int(self):
        return self.year*12+self.month

In [5]:
a = 10000
b = 1000
startDate = "01/2019"
endDate = "08/2019"
def parse(s):
    parts = s.split('/')
    if len(parts) == 2:
        year=int(parts[1])
        month=int(parts[0])
        return Date(month=month, year=year)
    else:
        month,day,year = parts
        return Date(month=month,day=day, year=year)

start_date = parse(startDate)
end_date = parse(endDate)
start_date

Date(month=1, year=2019, day=None)

In [7]:
history = [
    "$1000 payment received on 01/16/2019",
    "$2400 payment received on 03/20/2019",
    "$4000 payment received on 04/25/2019",
    "$3300 payment received on 05/06/2019",
    "$1500 payment received on 08/15/2019"
]
def parse_history2():
    history2 = dict()
    for x in history:
        payment, b = re.search('\$(\d+).*on (\d+/\d+/\d+)', x).groups()
        d = parse(b)
        history2[d.to_year_month_int()] = dict(payment=int(payment),on_time=d.day<=15, date=d)
    return history2
    
history2 = parse_history2()
history2

{24229: {'payment': 1000,
  'on_time': False,
  'date': Date(month=1, year=2019, day=16)},
 24231: {'payment': 2400,
  'on_time': False,
  'date': Date(month=3, year=2019, day=20)},
 24232: {'payment': 4000,
  'on_time': False,
  'date': Date(month=4, year=2019, day=25)},
 24233: {'payment': 3300,
  'on_time': True,
  'date': Date(month=5, year=2019, day=6)},
 24236: {'payment': 1500,
  'on_time': True,
  'date': Date(month=8, year=2019, day=15)}}

In [31]:
def last_n_months(date, n):
    r = []
    for i in range(0,n):
        x = history2.get(cur_date.to_year_month_int() - i)
        r.append(x)
    return r

# list(last_n_months(cur_date, 3))


In [32]:
cur_date

Date(month=9, year=2019, day=None)

In [40]:
servers = 3
list(zip(range(servers),[]*servers))

[]

In [44]:
range(servers),[[]for _ in range(servers)]

(range(0, 3), [[], [], []])

In [46]:
np.argmax([0]*10)

0

In [39]:
a= list('abcdefghijklmnopqrstuvwxyz')
m = dict()
# b='abcdefghijklmnopqrstuvwxyz'
# assert a==b


In [35]:
cur_date = parse(startDate)
b = b

while cur_date.month <= end_date.month:
    b_start = b
    months_elapsed = cur_date.to_year_month_int() - start_date.to_year_month_int() + 1
    # print('start cur_Date', cur_date)
    # print('months_elapsed', months_elapsed)

    # print('last 3 months')
    # If the user has paid less than 3·b dollars during the last three months,
    hists3mo = list(last_n_months(cur_date, 3))
    # print('hists', hists3mo)

    paid = sum(h['payment'] for h in hists3mo if h)
    n_late = sum(1 for h in hists3mo if h is None or ~h['on_time'] )

    # last two months
    if months_elapsed == 3:
        paid_enough = paid >= 3*b and n_late != 3
    else:
        paid_enough = True

    if months_elapsed == 2:
        didnt_pay_last_two_months = hists3mo[-2:] is [None, None]


    msg = f'Leave as it is, {b}'
    if not paid_enough or didnt_pay_last_2_months:
        b = round(b + a * d / 100)
        msg = f'Add one year, {b}'
    else:
        if months_elapsed == 3:
            # did we pay last 3 months?
            payments = [h['payment'] for h in hists3mo if h]
            if len(payments) == 3 and all(p>=b*2 for p in payments):
                b = round(b - a * d / 100)
                msg = f'Remove one year, {b}'

    
    
    pprint(dict(cur_date=cur_date,
                hists=hists3mo,
                paid=paid, 
                _late=n_late, 
                paid_enough=paid_enough,
                b_start=b,
                _3b=3*b,
                b=b,
                msg=msg,
                didnt_pay_last_2_months=didnt_pay_last_2_months,
                months_elapsed=months_elapsed), indent=2)
    
    
    

    cur_date = cur_date.inc_month()
    print()

{ '_3b': 3000,
  '_late': 3,
  'b': 1000,
  'b_start': 1000,
  'cur_date': Date(month=1, year=2019, day=None),
  'didnt_pay_last_2_months': False,
  'hists': [ { 'date': Date(month=1, year=2019, day=16),
               'on_time': False,
               'payment': 1000},
             None,
             None],
  'months_elapsed': 1,
  'msg': 'Leave as it is, 1000',
  'paid': 1000,
  'paid_enough': True}

{ '_3b': 3000,
  '_late': 3,
  'b': 1000,
  'b_start': 1000,
  'cur_date': Date(month=2, year=2019, day=None),
  'didnt_pay_last_2_months': False,
  'hists': [ None,
             { 'date': Date(month=1, year=2019, day=16),
               'on_time': False,
               'payment': 1000},
             None],
  'months_elapsed': 2,
  'msg': 'Leave as it is, 1000',
  'paid': 1000,
  'paid_enough': True}



NameError: name 'd' is not defined